# 🔐 Encrypto-Anonymizer

**A simple Google Colab-based tool by Elizabeth Shamblin to anonymize and encrypt CSV data securely.**

## 🔒 About the Encrypto-Anonymizer
- Uploaded files are stored **temporarily** on the virtual machine running your Colab session.
- Only **you** (the logged-in Google account) have access to them during that session.
- Files are **not shared** with Google, the public, or other users unless you explicitly do so.
- Colab does **not store** those files persistently — they are deleted when the session ends (e.g., if idle for 90 minutes or closed).


## 📁 Step 1: Upload a CSV file

In [ ]:
from google.colab import files
import pandas as pd
import io

uploaded = files.upload()
df = pd.read_csv(io.BytesIO(list(uploaded.values())[0]))
df.head()

## 🎯 Step 2: Select columns to anonymize

In [ ]:
columns_to_anonymize = ['Name', 'Email', 'ID']  # Change as needed
print("Selected columns to anonymize:", columns_to_anonymize)

## 🔀 Step 3: Anonymize and create encrypted mapping

In [ ]:
import uuid
from cryptography.fernet import Fernet
import json

# Generate a secure encryption key
fernet_key = Fernet.generate_key()
fernet = Fernet(fernet_key)

mapping = {}
anon_df = df.copy()

for col in columns_to_anonymize:
    new_ids = []
    for val in df[col]:
        anon_id = str(uuid.uuid4())
        encrypted_val = fernet.encrypt(str(val).encode()).decode()
        mapping[anon_id] = encrypted_val
        new_ids.append(anon_id)
    anon_df[col] = new_ids

anon_df.to_csv("anonymized.csv", index=False)
with open("mapping.json", "w") as f:
    json.dump(mapping, f)
with open("key.key", "wb") as f:
    f.write(fernet_key)
print("✅ Done anonymizing.")

## 💾 Step 4: Download anonymized and encrypted files

In [ ]:
files.download("anonymized.csv")
files.download("mapping.json")
files.download("key.key")

## 🔓 Step 5: Upload anonymized file + mapping + key to de-anonymize

In [ ]:
uploaded = files.upload()
anon_df = pd.read_csv("anonymized.csv")
with open("mapping.json") as f:
    mapping = json.load(f)
with open("key.key", "rb") as f:
    fernet = Fernet(f.read())

## 🔁 Step 6: De-anonymize and restore original values

In [ ]:
restored_df = anon_df.copy()
for col in columns_to_anonymize:
    original_values = []
    for anon_id in anon_df[col]:
        decrypted_val = fernet.decrypt(mapping[anon_id].encode()).decode()
        original_values.append(decrypted_val)
    restored_df[col] = original_values

restored_df.to_csv("restored.csv", index=False)
print("✅ Restoration complete.")

## 📥 Step 7: Download the restored file

In [ ]:
files.download("restored.csv")